In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Dropout,BatchNormalization, Conv2D, Input, Activation, MaxPooling2D, ZeroPadding2D
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import tensorflow.keras
from tensorflow.keras.layers import add

In [6]:
# create model
input_img = Input(shape=(224, 224, 3), name='main_input')

#VGG Net
x1 = Conv2D(64, (3, 3))(input_img)
x1 = Activation('relu')(x1)
x1 = Conv2D(64, (3, 3))(x1)
x1 = Activation('relu')(x1)
x1 = MaxPooling2D()(x1)
x1 = Conv2D(64, (3, 3))(x1)
x1 = Activation('relu')(x1)
x1 = Conv2D(64, (3, 3))(x1)
x1 = Activation('relu')(x1)
x1 = MaxPooling2D()(x1)
x1 = Conv2D(64, (3, 3))(x1)
x1 = Activation('relu')(x1)
x1 = MaxPooling2D()(x1)
x1 = Flatten()(x1)
x1 = Dense(256)(x1)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)
x1 = Dense(256)(x1)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)
 
#Res Net
x = Conv2D(64, (3, 3))(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = (ZeroPadding2D((1,1)))(x)
x = Conv2D(64, (3, 3))(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(1, (3, 3))(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = (ZeroPadding2D((1,1)))(x)
x = add([x, input_img])
x = Flatten()(x)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
 
x = tensorflow.keras.layers.concatenate([x1, x])
print(type(x))
out = Dense(1, activation='sigmoid')(x)
 
# Compile model
model = Model(inputs=input_img, outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>


In [10]:
model.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 222, 222, 64) 1792        main_input[0][0]                 
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 222, 222, 64) 0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 220, 220, 64) 36928       activation_10[0][0]              
______________________________________________________________________________________________

In [12]:
###########################################################
# 데이터 전처리
###########################################################
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(rescale=1./255)  # 모든 이미지를 1/255로 스케일을 조정합니다
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/train2"
validation_dir = './dataset/vali2'

batch_size = 16
# batch_size = 64


train_generator = train_datagen.flow_from_directory( \
        train_dir,                  # 타깃 디렉터리
        target_size=(224, 224),     # 모든 이미지를 224 × 224 크기
        batch_size=batch_size,
        class_mode='binary')        #

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 802 images belonging to 2 classes.
Found 286 images belonging to 2 classes.


In [13]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)  #(20, 150, 150, 3)
    print('배치 레이블 크기:', labels_batch.shape) #(20,)
    break

배치 데이터 크기: (16, 224, 224, 3)
배치 레이블 크기: (16,)


In [14]:
steps_per_epochs = 50 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 30

history = model.fit(
      train_generator ,
      steps_per_epoch=steps_per_epochs  ,
      epochs=epochs ,
      validation_data=validation_generator,
      validation_steps=18)

Epoch 1/30
50/50 [==============================] - 23s 310ms/step - loss: 0.4589 - accuracy: 0.7913 - val_loss: 1.0040 - val_accuracy: 0.5280
Epoch 2/30
50/50 [==============================] - 12s 247ms/step - loss: 0.1971 - accuracy: 0.9224 - val_loss: 1.2284 - val_accuracy: 0.5000
Epoch 3/30
50/50 [==============================] - 13s 249ms/step - loss: 0.0956 - accuracy: 0.9771 - val_loss: 1.3686 - val_accuracy: 0.5000
Epoch 4/30
50/50 [==============================] - 13s 254ms/step - loss: 0.0592 - accuracy: 0.9809 - val_loss: 1.5899 - val_accuracy: 0.4755
Epoch 5/30
50/50 [==============================] - 13s 259ms/step - loss: 0.0306 - accuracy: 0.9962 - val_loss: 1.3246 - val_accuracy: 0.4965
Epoch 6/30
50/50 [==============================] - 13s 259ms/step - loss: 0.0694 - accuracy: 0.9746 - val_loss: 2.5669 - val_accuracy: 0.4755
Epoch 7/30
50/50 [==============================] - 13s 260ms/step - loss: 0.0754 - accuracy: 0.9720 - val_loss: 1.9964 - val_accuracy: 0.5245

In [15]:
model.save("./tn_model/vgg_res_v4.h5")

D:\Program Files\python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
